In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from CVUSA_dataset import CVUSA_dataset_cropped, CVUSA_Dataset_Eval
from custom_models import ResNet, VIT, CLIP_model
from losses import SoftTripletBiLoss
from train import train
from eval import predict, accuracy, calculate_scores
from pytorch_metric_learning import losses as LS
from helper_func import get_rand_id, hyparam_info, save_losses
from transformers import CLIPProcessor




# data_path = '/media/fahimul/2B721C03261BDC8D/Research/datasets/CVUSA' #don't include the / at the end
# data_path = '/home/fa947945/datasets/CVUSA_Cropped/CVUSA' #don't include the / at the end
data_path = '/data/Research/Dataset/CVUSA_Cropped/CVUSA' #don't include the / at the end

train_data= pd.read_csv(f'{data_path}/splits/train-19zl.csv')
val_data= pd.read_csv(f'{data_path}/splits/val-19zl.csv')

# df_loss = pd.DataFrame(columns=['Loss'])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
# transform = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


train_ds = CVUSA_dataset_cropped(df = train_data, path=data_path, transform=transform)
val_que = CVUSA_Dataset_Eval(data_folder=data_path, split='train', img_type='query', transforms=transform)
val_ref = CVUSA_Dataset_Eval(data_folder=data_path, split='train', img_type='reference', transforms=transform)





def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")
    embed_dim = 256
    lr = 0.001
    batch_size = 64
    epochs = 500
    expID = get_rand_id()
    loss_margin = 1


    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
    val_loader_que = DataLoader(val_que, batch_size=batch_size, shuffle=False)
    val_loader_ref = DataLoader(val_ref, batch_size=batch_size, shuffle=False)

    # model = ResNet(emb_dim=embed_dim).to(device)
    # model_r = ResNet(emb_dim=embed_dim).to(device)
    # model_q = ResNet(emb_dim=embed_dim).to(device)

    # model = VIT().to(device)
    model = CLIP_model()
    
    # criterion = TripletLoss(margin=loss_margin)
    # criterion = nn.TripletMarginLoss(margin=0.5)
    criterion = SoftTripletBiLoss()

    


    parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    # for name, param in model.named_parameters():
    #     if param.requires_grad:
    #         print(name)
    optimizer = optim.Adam(parameters, lr=lr)
    # optimizer = optim.AdamW(parameters, lr=lr)
    # optimizer = optim.SGD(parameters, lr=lr)

    
    
    hyparam_info(emb_dim=embed_dim, loss_id=expID, ln_rate=lr, batch=batch_size, epc=epochs, ls_mrgn=loss_margin, trn_sz=train_data.shape[0], mdl_nm=model.modelName)

    print("Training Start")
    all_loses = train(model, criterion, optimizer, train_loader, num_epochs=epochs, dev=device)
    df_loss = pd.DataFrame({'Loss': all_loses})
    df_loss.to_csv(f'losses/losses_{expID}.csv')


    print("\nExtract Features:")
    query_features, query_labels = predict(model=model, dataloader=val_loader_que, dev=device, isQuery=True)
    reference_features, reference_labels = predict(model = model, dataloader=val_loader_ref, dev=device, isQuery=False) 
    
    print(query_labels)


    print("Compute Scores:")
    # r1 =  calculate_scores(query_features, reference_features, query_labels, reference_labels, step_size=1000, ranks=[1, 5, 10])
    r1 =  accuracy(query_features=query_features, reference_features=reference_features, query_labels=query_labels, topk=[1, 5, 10])
    
    save_losses(df=df_loss, 
                emb_dim=embed_dim, 
                loss_id=expID, 
                ln_rate=lr, 
                batch=batch_size, 
                epc=epochs, 
                ls_mrgn=loss_margin, 
                trn_sz=train_data.shape[0],
                mdl_nm=model.modelName,
                rslt=r1)


    print(r1) 
        







if __name__ == '__main__':
    main()


In [ ]:
from datetime import datetime
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import time
# from torch.cuda.amp import autocast
import numpy as np
from CVUSA_dataset import CVUSA_dataset_cropped, CVUSA_Dataset_Eval
# from CVUSA_dataset import CVUSA_Dataset_Eval
from custom_models import ResNet, VIT, CLIP_model
from losses import SoftTripletBiLoss, InfoNCE
from train import train
from eval import predict, accuracy, calculate_scores
import torch.nn.functional as F
import copy
import math
from pytorch_metric_learning import losses as LS
from helper_func import get_rand_id, hyparam_info, save_losses
from transformers import CLIPProcessor




# data_path = '/media/fahimul/2B721C03261BDC8D/Research/datasets/CVUSA' #don't include the / at the end
# data_path = '/home/fa947945/datasets/CVUSA_Cropped/CVUSA' #don't include the / at the end
data_path = '/data/Research/Dataset/CVUSA_Cropped/CVUSA' #don't include the / at the end

train_data= pd.read_csv(f'{data_path}/splits/train-19zl.csv')
val_data= pd.read_csv(f'{data_path}/splits/val-19zl.csv')

# df_loss = pd.DataFrame(columns=['Loss'])

transform = transforms.Compose([
    # transforms.Resize((224, 224)),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225]),
])
# transform = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


train_ds = CVUSA_dataset_cropped(df = train_data, path=data_path, transform=transform)
val_que = CVUSA_Dataset_Eval(data_folder=data_path, split='val', img_type='query', transforms=transform)
val_ref = CVUSA_Dataset_Eval(data_folder=data_path, split='val', img_type='reference', transforms=transform)





def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")
    embed_dim = 1000
    lr = 0.004
    batch_size = 64
    epochs = 100
    expID = get_rand_id()
    loss_margin = 1


    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
    val_loader_que = DataLoader(val_que, batch_size=batch_size, shuffle=False)
    val_loader_ref = DataLoader(val_ref, batch_size=batch_size, shuffle=False)

    # model = ResNet(emb_dim=embed_dim).to(device)
    # model_r = ResNet(emb_dim=embed_dim).to(device)
    # model_q = ResNet(emb_dim=embed_dim).to(device)

    # model = ResNet(emb_dim=embed_dim).to(device)
    # model = VIT().to(device)
    model = CLIP_model(embed_dim=embed_dim)
    
    # torch.save(model, f'model_weights/model_st.pth')

    # criterion = TripletLoss(margin=loss_margin)
    # criterion = nn.TripletMarginLoss(margin=0.5)
  
    # criterion = SoftTripletBiLoss()

    loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
    criterion = InfoNCE(loss_function=loss_fn,
                            device=device,
                            )


    parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    # for name, param in model.named_parameters():
    #     if param.requires_grad:
    #         print(name)
    optimizer = optim.Adam(parameters, lr=lr)
    # optimizer = optim.AdamW(parameters, lr=lr)
    # optimizer = optim.SGD(parameters, lr=lr)

    
    
    hyparam_info(emb_dim=embed_dim, loss_id=expID, ln_rate=lr, batch=batch_size, epc=epochs, ls_mrgn=loss_margin, trn_sz=train_data.shape[0], mdl_nm=model.modelName)

    print("Training Start")
    all_loses = train(model, criterion, optimizer, train_loader, num_epochs=epochs, dev=device)
    df_loss = pd.DataFrame({'Loss': all_loses})
    df_loss.to_csv(f'losses/losses_{expID}.csv')


    torch.save(model, f'model_weights/model_{expID}.pt')

    print("\nExtract Features:")
    query_features, query_labels = predict(model=model, dataloader=val_loader_que, dev=device, isQuery=True)
    reference_features, reference_labels = predict(model = model, dataloader=val_loader_ref, dev=device, isQuery=False) 
    


    print("Compute Scores:")
    # r1 =  calculate_scores(query_features, reference_features, query_labels, reference_labels, step_size=1000, ranks=[1, 5, 10])
    r1 =  accuracy(query_features=query_features, reference_features=reference_features, query_labels=query_labels, topk=[1, 5, 10])
    
    save_losses(df=df_loss, 
                emb_dim=embed_dim, 
                loss_id=expID, 
                ln_rate=lr, 
                batch=batch_size, 
                epc=epochs, 
                ls_mrgn=loss_margin, 
                trn_sz=train_data.shape[0],
                mdl_nm=model.modelName,
                rslt=r1)


    print(r1) 
        







if __name__ == '__main__':
    main()


In [ ]:
from datetime import datetime
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import time
# from torch.cuda.amp import autocast
import numpy as np
from CVUSA_dataset import CVUSA_dataset_cropped, CVUSA_Dataset_Eval
# from CVUSA_dataset import CVUSA_Dataset_Eval
from custom_models import ResNet, VIT, CLIP_model
from losses import SoftTripletBiLoss, InfoNCE
from train import train
from eval import predict, accuracy, calculate_scores
import torch.nn.functional as F
import copy
import math
from pytorch_metric_learning import losses as LS
from helper_func import get_rand_id, hyparam_info, save_losses
from transformers import CLIPProcessor




# data_path = '/media/fahimul/2B721C03261BDC8D/Research/datasets/CVUSA' #don't include the / at the end
# data_path = '/home/fa947945/datasets/CVUSA_Cropped/CVUSA' #don't include the / at the end
data_path = '/data/Research/Dataset/CVUSA_Cropped/CVUSA' #don't include the / at the end

train_data= pd.read_csv(f'{data_path}/splits/train-19zl.csv')
val_data= pd.read_csv(f'{data_path}/splits/val-19zl.csv')

# df_loss = pd.DataFrame(columns=['Loss'])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225]),
])
# transform = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


train_ds = CVUSA_dataset_cropped(df = train_data, path=data_path, transform=transform)
val_que = CVUSA_Dataset_Eval(data_folder=data_path, split='train', img_type='query', transforms=transform)
val_ref = CVUSA_Dataset_Eval(data_folder=data_path, split='train', img_type='reference', transforms=transform)





def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")
    embed_dim = 1000
    lr = 0.005
    batch_size = 64
    epochs = 50
    expID = get_rand_id()
    loss_margin = 1


    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
    val_loader_que = DataLoader(val_que, batch_size=batch_size, shuffle=False)
    val_loader_ref = DataLoader(val_ref, batch_size=batch_size, shuffle=False)

    # model = ResNet(emb_dim=embed_dim).to(device)
    # model_r = ResNet(emb_dim=embed_dim).to(device)
    # model_q = ResNet(emb_dim=embed_dim).to(device)

    # model = ResNet()
    # model = VIT().to(device)
    model = CLIP_model(embed_dim=embed_dim)
    
    # torch.save(model, f'model_weights/model_st.pth')

    # criterion = TripletLoss(margin=loss_margin)
    # criterion = nn.TripletMarginLoss(margin=0.5)
  
    # criterion = SoftTripletBiLoss()

    loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
    criterion = InfoNCE(loss_function=loss_fn,
                            device=device,
                            )


    parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    # for name, param in model.named_parameters():
    #     if param.requires_grad:
    #         print(name)
    optimizer = optim.Adam(parameters, lr=lr)
    # optimizer = optim.AdamW(parameters, lr=lr)
    # optimizer = optim.SGD(parameters, lr=lr)

    
    
    hyparam_info(emb_dim=embed_dim, loss_id=expID, ln_rate=lr, batch=batch_size, epc=epochs, ls_mrgn=loss_margin, trn_sz=train_data.shape[0], mdl_nm=model.modelName)

    print("Training Start")
    all_loses = train(model, criterion, optimizer, train_loader, num_epochs=epochs, dev=device)
    df_loss = pd.DataFrame({'Loss': all_loses})
    df_loss.to_csv(f'losses/losses_{expID}.csv')



    print("\nExtract Features:")
    query_features, query_labels = predict(model=model, dataloader=val_loader_que, dev=device, isQuery=True)
    reference_features, reference_labels = predict(model = model, dataloader=val_loader_ref, dev=device, isQuery=False) 
    


    print("Compute Scores:")
    # r1 =  calculate_scores(query_features, reference_features, query_labels, reference_labels, step_size=1000, ranks=[1, 5, 10])
    r1 =  accuracy(query_features=query_features, reference_features=reference_features, query_labels=query_labels, topk=[1, 5, 10])
    
    save_losses(df=df_loss, 
                emb_dim=embed_dim, 
                loss_id=expID, 
                ln_rate=lr, 
                batch=batch_size, 
                epc=epochs, 
                ls_mrgn=loss_margin, 
                trn_sz=train_data.shape[0],
                mdl_nm=model.modelName,
                rslt=r1)


    print(r1) 
        







if __name__ == '__main__':
    main()


In [8]:
from datetime import datetime
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import time
# from torch.cuda.amp import autocast
import numpy as np
from CVUSA_dataset import CVUSA_dataset_cropped, CVUSA_Dataset_Eval
# from CVUSA_dataset import CVUSA_Dataset_Eval
from custom_models import ResNet, VIT, CLIP_model
from losses import Contrastive_loss, SoftTripletBiLoss, InfoNCE
from train import train
from eval import predict, accuracy, calculate_scores
import torch.nn.functional as F
import copy
import math
from pytorch_metric_learning import losses as LS
from helper_func import get_rand_id, hyparam_info, save_losses
from transformers import CLIPProcessor




# data_path = '/media/fahimul/2B721C03261BDC8D/Research/datasets/CVUSA' #don't include the / at the end
# data_path = '/home/fa947945/datasets/CVUSA_Cropped/CVUSA' #don't include the / at the end
data_path = '/data/Research/Dataset/CVUSA_Cropped/CVUSA' #don't include the / at the end

# train_data= pd.read_csv(f'{data_path}/splits/train-19zl.csv')
train_data= pd.read_csv(f'{data_path}/splits/train-19zl_30.csv')
val_data= pd.read_csv(f'{data_path}/splits/val-19zl.csv')

# df_loss = pd.DataFrame(columns=['Loss'])

transform = transforms.Compose([
    # transforms.Resize((224, 224)),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225]),
])


train_ds = CVUSA_dataset_cropped(df = train_data, path=data_path, transform=transform)
val_que = CVUSA_Dataset_Eval(data_folder=data_path, split='train', img_type='query', transforms=transform)
val_ref = CVUSA_Dataset_Eval(data_folder=data_path, split='train', img_type='reference', transforms=transform)





def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")
    embed_dim = 1000
    lr = 0.00001
    batch_size = 99
    epochs = 50
    expID = get_rand_id()
    loss_margin = 1


    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
    val_loader_que = DataLoader(val_que, batch_size=batch_size, shuffle=False)
    val_loader_ref = DataLoader(val_ref, batch_size=batch_size, shuffle=False)

    # os.mkdir(f'model_weights/{expID}')

    # model = ResNet(emb_dim=embed_dim).to(device)
    # model_r = ResNet(emb_dim=embed_dim).to(device)
    # model_q = ResNet(emb_dim=embed_dim).to(device)

    # model = ResNet()
    # model = VIT().to(device)
    # model = CLIP_model(embed_dim=embed_dim)







    # torch.save(model, f'model_weights/{expID}/model_st.pth')
    
    
    m_path = f'model_weights/{6311184}/model_tr.pth'
    model = torch.load(m_path)
    

    # criterion = TripletLoss(margin=loss_margin)
    # criterion = nn.TripletMarginLoss(margin=0.5)
  
    # criterion = SoftTripletBiLoss()

    loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
    criterion = InfoNCE(loss_function=loss_fn,
                            device=device,
                            )



    parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    # for name, param in model.named_parameters():
    #     if param.requires_grad:
    #         print(name)
    # optimizer = optim.Adam(parameters, lr=lr)
    optimizer = optim.AdamW(parameters, lr=lr)
    # optimizer = optim.SGD(parameters, lr=lr)

    
    
    hyparam_info(emb_dim=embed_dim, loss_id=expID, ln_rate=lr, batch=batch_size, epc=epochs, ls_mrgn=loss_margin, trn_sz=train_data.shape[0], mdl_nm=model.modelName)

    # print("Training Start")
    # all_loses = train(model, criterion, optimizer, train_loader, num_epochs=epochs, dev=device)
    # df_loss = pd.DataFrame({'Loss': all_loses})
    # df_loss.to_csv(f'losses/losses_{expID}.csv')



    print("\nExtract Features:")
    query_features, query_labels = predict(model=model, dataloader=val_loader_que, dev=device, isQuery=True)
    reference_features, reference_labels = predict(model = model, dataloader=val_loader_ref, dev=device, isQuery=False) 
    


    print("Compute Scores:")
    # r1 =  calculate_scores(query_features, reference_features, query_labels, reference_labels, step_size=1000, ranks=[1, 5, 10])
    r1 =  accuracy(query_features=query_features, reference_features=reference_features, query_labels=query_labels, topk=[1, 5, 10])
    
    # save_losses(df=df_loss, 
    #             emb_dim=embed_dim, 
    #             loss_id=expID, 
    #             ln_rate=lr, 
    #             batch=batch_size, 
    #             epc=epochs, 
    #             ls_mrgn=loss_margin, 
    #             trn_sz=train_data.shape[0],
    #             mdl_nm=model.modelName,
    #             rslt=r1)


    print(r1) 
        

    # torch.save(model, f'model_weights/{expID}/model_tr.pth')






if __name__ == '__main__':
    main()


/home/fahimul/.conda/envs/CVGL/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/fahimul/.conda/envs/CVGL/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Device: cuda

Hyperparameter info:
Exp ID: 6401805
Embedded dimension: 1000
Learning rate: 1e-05
Batch Size: 99
Loss Margin: 1
Epoch: 50
Training Size: 10658
Model Name: ResNet18



Extract Features:


100%|██████████| 359/359 [04:09<00:00,  1.44it/s]


Compute Scores:
Percentage-top1:67.85432849262637, top5:84.80806033997523, top10:90.04559270516718, top1%:99.91275469998874, time:3.057422399520874
[67.85432849 84.80806034 90.04559271 99.9127547 ]


In [1]:
from datetime import datetime
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import time
# from torch.cuda.amp import autocast
import numpy as np
from CVUSA_dataset import CVUSA_dataset_cropped, CVUSA_Dataset_Eval
# from CVUSA_dataset import CVUSA_Dataset_Eval
from custom_models import ResNet, VIT, CLIP_model
from losses import Contrastive_loss, SoftTripletBiLoss, InfoNCE
from train import train
from eval import predict, accuracy, calculate_scores
import torch.nn.functional as F
import copy
import math
from pytorch_metric_learning import losses as LS
from helper_func import get_rand_id, hyparam_info, save_losses
from transformers import CLIPProcessor




# data_path = '/media/fahimul/2B721C03261BDC8D/Research/datasets/CVUSA' #don't include the / at the end
# data_path = '/home/fa947945/datasets/CVUSA_Cropped/CVUSA' #don't include the / at the end
data_path = '/data/Research/Dataset/CVUSA_Cropped/CVUSA' #don't include the / at the end

# train_data= pd.read_csv(f'{data_path}/splits/train-19zl.csv')
train_data= pd.read_csv(f'{data_path}/splits/train-19zl_30.csv')
val_data= pd.read_csv(f'{data_path}/splits/val-19zl.csv')

# df_loss = pd.DataFrame(columns=['Loss'])

transform = transforms.Compose([
    # transforms.Resize((224, 224)),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225]),
])


train_ds = CVUSA_dataset_cropped(df = train_data, path=data_path, transform=transform)
val_que = CVUSA_Dataset_Eval(data_folder=data_path, split='train', img_type='query', transforms=transform)
val_ref = CVUSA_Dataset_Eval(data_folder=data_path, split='train', img_type='reference', transforms=transform)





def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")
    embed_dim = 1000
    lr = 0.00001
    batch_size = 64
    epochs = 100
    expID = get_rand_id()
    loss_margin = 1


    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
    val_loader_que = DataLoader(val_que, batch_size=batch_size, shuffle=False)
    val_loader_ref = DataLoader(val_ref, batch_size=batch_size, shuffle=False)

    # os.mkdir(f'model_weights/{expID}')

    # model = ResNet(emb_dim=embed_dim).to(device)
    # model_r = ResNet(emb_dim=embed_dim).to(device)
    # model_q = ResNet(emb_dim=embed_dim).to(device)

    # model = ResNet().to(device)
    # model = VIT().to(device)
    model = CLIP_model(embed_dim=embed_dim)
    
    # torch.save(model, f'model_weights/{expID}/model_st.pth')

    # criterion = TripletLoss(margin=loss_margin)
    # criterion = nn.TripletMarginLoss(margin=0.5)
  
    # criterion = SoftTripletBiLoss()

    loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.5)
    criterion = InfoNCE(loss_function=loss_fn,
                            device=device,
                            )



    parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    # for name, param in model.named_parameters():
    #     if param.requires_grad:
    #         print(name)
    optimizer = optim.Adam(parameters, lr=lr)
    # optimizer = optim.AdamW(parameters, lr=lr)
    # optimizer = optim.SGD(parameters, lr=lr)

    
    
    hyparam_info(emb_dim=embed_dim, loss_id=expID, ln_rate=lr, batch=batch_size, epc=epochs, ls_mrgn=loss_margin, trn_sz=train_data.shape[0], mdl_nm=model.modelName)

    print("Training Start")
    all_loses = train(model, criterion, optimizer, train_loader, num_epochs=epochs, dev=device)
    df_loss = pd.DataFrame({'Loss': all_loses})
    df_loss.to_csv(f'losses/losses_{expID}.csv')



    print("\nExtract Features:")
    query_features, query_labels = predict(model=model, dataloader=val_loader_que, dev=device, isQuery=True)
    reference_features, reference_labels = predict(model = model, dataloader=val_loader_ref, dev=device, isQuery=False) 
    


    print("Compute Scores:")
    # r1 =  calculate_scores(query_features, reference_features, query_labels, reference_labels, step_size=1000, ranks=[1, 5, 10])
    r1 =  accuracy(query_features=query_features, reference_features=reference_features, query_labels=query_labels, topk=[1, 5, 10])
    
    save_losses(df=df_loss, 
                emb_dim=embed_dim, 
                loss_id=expID, 
                ln_rate=lr, 
                batch=batch_size, 
                epc=epochs, 
                ls_mrgn=loss_margin, 
                trn_sz=train_data.shape[0],
                mdl_nm=model.modelName,
                rslt=r1)


    print(r1) 
        

    # torch.save(model, f'model_weights/{expID}/model_tr.pth')






if __name__ == '__main__':
    main()


/home/fahimul/.conda/envs/CVGL/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/fahimul/.conda/envs/CVGL/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/fahimul/.conda/envs/CVGL/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Device: cuda


/home/fahimul/.conda/envs/CVGL/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/fahimul/.conda/envs/CVGL/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()



Hyperparameter info:
Exp ID: 6257958
Embedded dimension: 1000
Learning rate: 1e-05
Batch Size: 64
Loss Margin: 1
Epoch: 100
Training Size: 10658
Model Name: CLIP


Training Start

Date: 2024-05-20 22:19:19.471950

Epoch#0


100%|██████████| 167/167 [06:45<00:00,  2.43s/it]


...

Epoch: 99/100 Loss: 2.989344596862793
Epoch#99


100%|██████████| 167/167 [03:58<00:00,  1.43s/it]


Epoch: 100/100 Loss: 2.9894866943359375

Date: 2024-05-21 07:31:15.466920


Extract Features:


100%|██████████| 556/556 [04:37<00:00,  2.00it/s]


Compute Scores:
Percentage-top1:18.445345041089723, top5:25.450298322638748, top10:28.06484295845998, top1%:49.40898345153664, time:2.6157195568084717
[18.44534504 25.45029832 28.06484296 49.40898345]
